In [2]:

!pip install -q transformers accelerate peft datasets bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.2 MB/s eta 0:00:00


In [3]:
!pip install -U datasets transformers peft huggingface-hub fsspec


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 37.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.34.1
    Uninstalling huggingface-hub-0.34.1:
      Successfully uninstalled huggingface-hub-0.34.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.54.0
    Uninstalling transformers-4.54.0:
      Successfully uninstalled transformers-4.54.0
  Attempting uninstall: peft
    Found existing installation: peft 0.16.0
    Uninstalling peft-0.16.0:
      Successfully uninstalled peft-0.16.0


In [4]:
from datasets import load_dataset

dataset = load_dataset("lex_glue", "ledgar", split = "train") # loading the ledgar dataset from the lex glue benchmark

print(dataset.features) #print column names
print(dataset[0]) #inspecting the first data values

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.31M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/3.44M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

{'text': Value('string'), 'label': ClassLabel(names=['Adjustments', 'Agreements', 'Amendments', 'Anti-Corruption Laws', 'Applicable Laws', 'Approvals', 'Arbitration', 'Assignments', 'Assigns', 'Authority', 'Authorizations', 'Base Salary', 'Benefits', 'Binding Effects', 'Books', 'Brokers', 'Capitalization', 'Change In Control', 'Closings', 'Compliance With Laws', 'Confidentiality', 'Consent To Jurisdiction', 'Consents', 'Construction', 'Cooperation', 'Costs', 'Counterparts', 'Death', 'Defined Terms', 'Definitions', 'Disability', 'Disclosures', 'Duties', 'Effective Dates', 'Effectiveness', 'Employment', 'Enforceability', 'Enforcements', 'Entire Agreements', 'Erisa', 'Existence', 'Expenses', 'Fees', 'Financial Statements', 'Forfeitures', 'Further Assurances', 'General', 'Governing Laws', 'Headings', 'Indemnifications', 'Indemnity', 'Insurances', 'Integration', 'Intellectual Property', 'Interests', 'Interpretations', 'Jurisdictions', 'Liens', 'Litigations', 'Miscellaneous', 'Modifications'

In [5]:
from datasets import load_dataset
import pandas as pd

labels = dataset.features['label'].names

def convert_instruction(example):
  return{
        "instruction": "Classify the legal clause below into a clause type.",
        "input": example["text"],
        "output": labels[example["label"]]

  }

formatted_dataset = dataset.map(convert_instruction, remove_columns= dataset.column_names)

sampled_dataset = formatted_dataset.select(range(1000))
sampled_dataset.to_json("ledgar_lora_sample.json", orient="records", lines=True)

print("✅ Done! Sampled dataset saved as `ledgar_lora_sample.json`.")


Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

✅ Done! Sampled dataset saved as `ledgar_lora_sample.json`.


In [6]:
from transformers import Trainer

class CausalLoraTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        input_ids = inputs["input_ids"]
        attention_mask = inputs.get("attention_mask", None)
        labels = inputs["labels"]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        return (loss, outputs) if return_outputs else loss


In [7]:
"""
This script fine-tunes the TinyLlama-1.1B-Chat model using LoRA (Low-Rank Adaptation)
on a subset of the LEDGAR legal clause classification dataset.

- The dataset is formatted as input-output instruction pairs in JSON
- Uses PEFT (parameter-efficient fine-tuning) with bitsandbytes 4-bit quantization
- Tokenizes data in TinyLlama instruction format
- Trains using Hugging Face Trainer and saves the LoRA adapter only
"""

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
from datasets import load_dataset
import torch

# Load dataset
dataset = load_dataset("json", data_files="ledgar_lora_sample.json", split="train")

# Base model config
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
bnb_config = BitsAndBytesConfig(load_in_4bit=True)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)

# Format input-output pair in TinyLlama chat format
def tokenise(x):
    prompt = f"""### Instruction:
Classify the legal clause below into a clause type.

### Input:
{x['input']}

### Response:
{x['output']}"""
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenised = dataset.map(tokenise, remove_columns=dataset.column_names)

# LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # for Llama-based models
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Collator and training args
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./lora-legal-tinyllama",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_total_limit=1,
    save_strategy="epoch",
    fp16=True,
    report_to="none",
)

# Use regular Trainer if you don't have custom CausalLoraTrainer
from transformers import Trainer

trainer = Trainer(
    model=model,
    train_dataset=tokenised,
    args=training_args,
    data_collator=data_collator,
)

# Train
trainer.train()

# ✅ Save LoRA adapter correctly
# Only PeftModel can generate adapter_config.json
if isinstance(trainer.model, PeftModel):
    trainer.model.save_pretrained("./lora-legal-tinyllama")
else:
    model.save_pretrained("./lora-legal-tinyllama")

tokenizer.save_pretrained("./lora-legal-tinyllama")


Generating train split: 0 examples [00:00, ? examples/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:457: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
10,1.852400
20,1.578600
30,1.394800
40,1.332100
50,1.265700
60,1.281800
70,1.216200
80,1.228100
90,1.199200
100,1.207100


('./lora-legal-tinyllama/tokenizer_config.json',
 './lora-legal-tinyllama/special_tokens_map.json',
 './lora-legal-tinyllama/chat_template.jinja',
 './lora-legal-tinyllama/tokenizer.model',
 './lora-legal-tinyllama/added_tokens.json',
 './lora-legal-tinyllama/tokenizer.json')

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch
import os

#Model setup
base_model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
lora_model_dir = "./lora-legal-tinyllama"  # LoRA output dir from training

# we want to check if our adapter exists, so for that we will first specify the path
adapter_config_path = os.path.join(lora_model_dir, "adapter_config.json")
if not os.path.exists(adapter_config_path):
    raise FileNotFoundError("LoRA adapter not found.")

# we want to carry out 4-bit loading  (quantization) to reduce memory usage and we can achieve this using the BitsAndBytesConfig from huggingface
bnb_config = BitsAndBytesConfig(load_in_4bit=True)

# now we want to use a pretrained tokenizer so we will use tinyllama
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto", # here, we are automatically placing our gpu/cpu
    quantization_config=bnb_config, # 4-bit quantization will be used as we had defined bnb_config as that
    trust_remote_code=True
)

# Apply LoRA
model = PeftModel.from_pretrained(base_model, lora_model_dir)
model.eval()

# === Few-shot prompt ===
few_shot_prompt = """### Instruction:
Explain the following legal clauses simply.

### Clause:
The Parties agree to keep the terms of this Agreement confidential and shall not disclose such terms to any third party.
### Explanation:
This means both sides must keep the contract terms private and not share them with anyone.

---

### Clause:
If either party breaches this Agreement, the non-breaching party may terminate the contract immediately.
### Explanation:
If one side breaks the rules, the other side can cancel the contract right away.

---

### Clause:
The Contractor shall indemnify the Client against all claims, damages, and expenses arising from the performance of the work.
### Explanation:
The contractor will cover any legal or financial trouble the client faces because of the contractor's work.

---

### Clause:
Upon written notice, either Party may assign this Agreement to an affiliate without consent of the other Party.
### Explanation:
Either side can transfer the contract to a related company without needing approval, as long as they give written notice.

---

### Clause:
The governing law of this Agreement shall be the laws of the State of California.
### Explanation:
This contract will follow California’s laws.

---

If the clause does not make legal sense, respond with:
"This clause does not appear to be a valid legal statement."

---

### Clause:
The Contractor shall indemnify the Client against all claims, damages, and expenses arising from the performance of the work.
### Explanation:"""

# === Inference ===
inputs = tokenizer(few_shot_prompt, return_tensors="pt").to(model.device) # we want to tokenize our input prompt an then we move it to the same device as our model

with torch.no_grad(): #Disable the gradient calculation so that memory usage is reduced
    outputs = model.generate(
        **inputs,
        max_new_tokens=100, # model should generate no more than 100 tokens
        do_sample=False, #disable sampling since we want to do beam search
        num_beams=3,
        early_stopping=True, #stop if all beams finish early
        pad_token_id=tokenizer.eos_token_id, #this is done to avoid pad errors
    )

#Decoding
generated_tokens = outputs[0][inputs['input_ids'].shape[-1]:]
result = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

# Extract the explanation only (first part before the next '### Clause' if present)
if "### Clause:" in result:
    result = result.split("### Clause:")[0].strip()

# If multiple lines, keep just the first line of explanation
if '\n' in result:
    result = result.split('\n')[0].strip()

print(f"Explanation: '{result}'")





Explanation: 'The contractor will cover any legal or financial trouble the client faces because of the contractor's work.'


In [18]:
!zip -r lora-legal-tinyllama.zip ./lora-legal-tinyllama


  adding: lora-legal-tinyllama/ (stored 0%)
  adding: lora-legal-tinyllama/tokenizer.model (deflated 55%)
  adding: lora-legal-tinyllama/adapter_config.json (deflated 56%)
  adding: lora-legal-tinyllama/special_tokens_map.json (deflated 73%)
  adding: lora-legal-tinyllama/tokenizer.json (deflated 85%)
  adding: lora-legal-tinyllama/checkpoint-189/ (stored 0%)
  adding: lora-legal-tinyllama/checkpoint-189/tokenizer.model (deflated 55%)
  adding: lora-legal-tinyllama/checkpoint-189/scaler.pt (deflated 60%)
  adding: lora-legal-tinyllama/checkpoint-189/adapter_config.json (deflated 56%)
  adding: lora-legal-tinyllama/checkpoint-189/special_tokens_map.json (deflated 73%)
  adding: lora-legal-tinyllama/checkpoint-189/trainer_state.json (deflated 74%)
  adding: lora-legal-tinyllama/checkpoint-189/optimizer.pt (deflated 8%)
  adding: lora-legal-tinyllama/checkpoint-189/tokenizer.json (deflated 85%)
  adding: lora-legal-tinyllama/checkpoint-189/scheduler.pt (deflated 56%)
  adding: lora-legal-

In [19]:
from google.colab import files
files.download("lora-legal-tinyllama.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>